In [1]:
import pandas as pd
import numpy as np
import csv
import os
from scipy import stats
import pyphysio as ph
from pdb import set_trace
import random
from datetime import timedelta, datetime
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

Please cite:
Bizzego et al. (2019) 'pyphysio: A physiological signal processing library for data science approaches in physiology', SoftwareX


In [2]:
#survey windows 
#these are available in the 10 Minute Windows subfolder
surv = pd.read_csv('/Users/sandoval/Library/CloudStorage/Box-Box/R15 Sensor Preprocessing and Analysis/10 Minute Windows/PR003/PR003 Survey Windows MERGED.csv',parse_dates=['timestamp'], infer_datetime_format=True, index_col=[0])

#baseline windows
#also available in the 10 Minute Windows subfolder
base = pd.read_csv('/Users/sandoval/Library/CloudStorage/Box-Box/R15 Sensor Preprocessing and Analysis/10 Minute Windows/PR003/PR003 Baseline MERGED.csv',parse_dates=['timestamp'], infer_datetime_format=True, index_col=[0])

#ema responses
#found in Sofie Data subfolder, may have to dig for it 
ema = pd.read_csv("/Users/sandoval/Library/CloudStorage/Box-Box/R15 Sensor Preprocessing and Analysis/Data/Sofie Data/PR/Maintained active dx/PR003/PR003_rawwithtime2022-07-03.csv", parse_dates=['ethica_time_utc'], infer_datetime_format=True, index_col=[0])

<ipython-input-2-6d888cb823e8>:3: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  surv = pd.read_csv('/Users/sandoval/Library/CloudStorage/Box-Box/R15 Sensor Preprocessing and Analysis/10 Minute Windows/PR003/PR003 Survey Windows MERGED.csv',parse_dates=['timestamp'], infer_datetime_format=True, index_col=[0])


FileNotFoundError: [Errno 2] No such file or directory: '/Users/sandoval/Library/CloudStorage/Box-Box/R15 Sensor Preprocessing and Analysis/10 Minute Windows/PR003/PR003 Survey Windows MERGED.csv'

In [ ]:
### these scripts are taken from prior scripts written by collaborators with some modification by me
### these are all the functions we need... i think

#Preprocessing filters that will be applied to raw data samples
def exp_moving_average(signal, w):
    """Expoential moving average filter from pandas"""
    return pd.Series(signal.ewm(span=w, adjust=True).mean(), signal.index)

def filt_EDA(df_data):
    """Apply filter to EDA signal using processing steps from Pyphysio.
    
    Parameters
    ----------
    df_data : pandas.DataFrame
        DataFrame with EDA time and signal data
    
    Returns
    -------
    pandas.DataFrame 
        Updated DataFrame with Tonic and Phasic signals
    
    """
    
    # Set up Pyphysio EvenlySignal object
    eda_data = ph.EvenlySignal(values = df_data['EDA'].to_numpy(), sampling_freq = 4, signal_type='EDA')
    
    # Apply IIR filter
    eda_data = ph.IIRFilter(fp=0.8, fs=1.1, ftype='ellip')(eda_data)
    driver = ph.DriverEstim()(eda_data)
    
    # Estimate Tonic and Phasic signals
    phasic, tonic, _ = ph.PhasicEstim(delta=0.02)(driver)
    
    # Adjust signal length to match length of original EDA signal
    if len(phasic) != len(eda_data.get_values()):
        phasic = np.append(phasic.get_values(), phasic[-1])
        tonic = np.append(tonic.get_values(), tonic[-1])    
    
    # Append signal data to DataFrame
    df_data.loc[:,'Tonic'] = tonic
    df_data.loc[:,'Phasic'] = phasic
    return df_data  

def filt_TEMP(df_data):
    """Apply filter to TEMP signal using predetermined values"""
    df_data['TEMP_Filtered'] = exp_moving_average(df_data['TEMP'],60)
    return df_data

def filter_signals(df_data):
    """Apply filters/processing to respective signals.
    
    This function is intended to be used in conjunction with the
    pandas.DataFrame.apply() method, which passes a column of a DataFrame at
    a time.
    
    Parameters
    ----------
    df_data : pandas.Series
        A Series of DataFrames which contain all the signals for a single
        session.
    
    Returns
    -------
    pandas.Series
        Updated Series of DataFrames containing filtered/processed signal
        data
        
    Notes
    -----
    No filtering/processing for ACC, HR, and IBI signals is implemented,
    but is commented out for implementation in the future.
    
    """
    
    #df_data.loc['BVP'].loc[:,'BVP'] = filt_BVP(df_data.loc['BVP'])
    df_data = filt_EDA(df_data)
    df_data = filt_TEMP(df_data)
    
    return df_data


#RMS calculation Helper Function
def rms(data):
    return np.sqrt(np.mean(data ** 2))

#Every other feature is easy to calculate using Python built-ins
def feature_extract(df_data):
    result = {}
    result['Time'] = df_data.loc[:,'timestamp'].min()
    #for featbase in ['HR','EDA','TEMP','Tonic','Phasic','TEMP_Filtered']:
    for featbase in ['HR','EDA','TEMP', 'meanCenteredEDA', 'meanCenteredHR', 'meanCenteredTEMP']:
        #set_trace()
        result[featbase + '_Mean'] = df_data.loc[:,featbase].mean()
        result[featbase + '_Minimum'] = df_data.loc[:,featbase].min()
        result[featbase + '_Maximum'] = df_data.loc[:,featbase].max()
        result[featbase + '_Stdev'] = df_data.loc[:,featbase].std()
        result[featbase + '_RMS'] = rms(df_data.loc[:,featbase])
        result[featbase + '_MAD'] = df_data.loc[:,featbase].mad()
        result[featbase + '_MAV'] = df_data.loc[:,featbase].abs().max()
        result[featbase + '_Median'] = df_data.loc[:,featbase].median()
        result[featbase + '_P25'] = df_data.loc[:,featbase].quantile(0.25)
        result[featbase + '_P75'] = df_data.loc[:,featbase].quantile(0.75)
    return pd.Series(result, dtype='object')


#simplified the feature extraction function to only grab mean and median
def average_calc(df_data):
    result = {}    
    result['EDA_Mean'] = df_data.loc[:,'EDA'].mean()
    result['EDA_Median'] = df_data.loc[:,'EDA'].median()
    result['HR_Mean'] = df_data.loc[:,'HR'].mean()
    result['HR_Median'] = df_data.loc[:,'HR'].median()
    result['TEMP_Mean'] = df_data.loc[:,'TEMP'].mean()
    result['TEMP_Median'] = df_data.loc[:,'TEMP'].median()

    return pd.Series(result, dtype='object')

#creating a function to match rows
#remember that the physio timestamp must ALWAYS BE BEFORE the EMA timestamp

def windowMatch(features,surveys): 
    matchlist = []
    for index, frow in features.iterrows():
        physiotime = frow['Time']
        for index, srow in surveys.iterrows():
            ematime = srow['ethica_time_utc']
            df3 = ematime - physiotime
            if df3 < delta and df3 > zeropoint:
                #print("Physio time (", physiotime, ") and EMA time (", ematime, ") difference is", df3)
                matchlist.append(pd.concat([frow,srow],axis=0))
    return pd.DataFrame(matchlist)

In [ ]:
#this chunk of code will normalize our date (deducting the average/median of the baseline windows from each datapoint in survey windows)

baseline_values = base.groupby(['code']).apply(average_calc) # calculating averages of the baseline

#creating variables to subtract from the dataframe columns
edamean = baseline_values.at[0,"EDA_Mean"]
edamed = baseline_values.at[0,"EDA_Median"]
hrmean = baseline_values.at[0,"HR_Mean"]
hrmed = baseline_values.at[0,"HR_Median"]
tempmean = baseline_values.at[0,"TEMP_Mean"]
tempmed = baseline_values.at[0,"TEMP_Median"]

#subtracting our means/medians from each value in the participant's data - excluding ACC 
surv['meanCenteredEDA'] = surv['EDA'] - edamean
surv['medianCenteredEDA'] = surv['EDA'] - edamed
surv['meanCenteredHR'] = surv['HR'] - hrmean
surv['medianCenteredHR'] = surv['HR'] - hrmed
surv['meanCenteredTEMP'] = surv['TEMP'] - tempmean
surv['medianCenteredTEMP'] = surv['TEMP'] - tempmed

surv.head()

In [ ]:
#this chunk is to prepare the physio data to be matched to timestamps 

physio = surv.sort_values(['timestamp'], ignore_index=True) #sort by timestamp
physio = physio.groupby(['event']).apply(feature_extract) #applying feature extraction to create summary features

#setting up a timedelta to base our window matching off of
#a survey window can never occur more than an hour from when a survey was completed, so our delta is 1 hour
#if a window is more than an hour separate from survey completion, it cannot be matched
#however, the survey window timestamp should ALWAYS precede the survey completion time
delta = timedelta(hours = 1)
zeropoint = timedelta(seconds = 1)

#now we apply the window matching function - it calculates the time differences of each timestamp
#if the time difference is less than 1 hour, match the windows together
mydf = windowMatch(physio, ema)
for index,row in mydf.iterrows():
    print (abs(row['ethica_time_utc']-row['Time']))
    
mydf.head()

In [ ]:
#putting together physio featureset to be ready for correlations
featbase =['HR','EDA','TEMP', 'meanCenteredEDA', 'meanCenteredHR', 'meanCenteredTEMP']
featstat =['_Mean','_Minimum','_Stdev','_RMS','_MAD','_MAV','_Median','_P25','_P75']
physio_feats = {}
allfeats = []
for fb in featbase:
    curfeats=[]
    for fs in featstat:
        curfeats.append(fb+fs)
        allfeats.append(fb+fs)
    physio_feats[fb] = mydf[curfeats]
physio_feats['All'] = mydf[allfeats]
        

In [ ]:
#centering EMA just in case - just hashtag out this loop if it's not necessary

for col in ema_feats:
    ema_feats[col + '_meanCentered'] = ema_feats[col] - ema_feats.loc[:,col].mean()
    ema_feats[col + '_medCentered'] = ema_feats[col] - ema_feats.loc[:,col].median()
    
    
pd.set_option('display.max_columns', None)
ema_feats

In [ ]:
#cleaning up EMA dataset for correlations
excl_list = ['ethica_time','lag','tdif','cumsumT','ethica_time_utc','dayvar','beepvar','beepconsec']
surfeat=[ele for ele in list(ema.columns) if ele not in excl_list]
ema_feats = mydf[surfeat]

In [ ]:
#now we're correlating physio with EMA here
corrdict = {}
for sur in surfeat:
    corrdict[sur]=physio_feats['All'].corrwith(ema_feats[sur],method='pearson')
    corrdict[sur]=physio_feats['All'].corrwith(ema_feats[sur],method='pearson')
corr_df = pd.DataFrame(corrdict)
                                                     
corr_df

In [ ]:
#just in case you want to do some scatterplots 
#you'll notice a lot of outliers in some of these plots - we may want to establish some way to remove outliers - perhaps if something is 4+ SDs from the mean?
ax1= mydf.plot.scatter(x='HR_Mean',y='restrict')

In [ ]:
#/Users/sandoval/Library/CloudStorage/Box-Box/R15 Sensor Preprocessing and Analysis/JPCS Special Issue/normalizing.ipynb

#this will save the centered survey windows
surv.to_csv("""/Users/sandoval/Library/CloudStorage/Box-Box/R15 Sensor Preprocessing and Analysis/JPCS Special Issue/Normalized Data/PR003 Survey Windows CENTERED.csv""")

#this will save our correlation matrix 
corr_df.to_csv("""/Users/sandoval/Library/CloudStorage/Box-Box/R15 Sensor Preprocessing and Analysis/JPCS Special Issue/Correlation Matrices/PR003 Survey Correlations.csv""")

